In [ ]:
# Initialize models
log_reg = LogisticRegression(max_iter=1000)
svm = SVC()
rf = RandomForestClassifier()


In [ ]:
log_reg.fit(X_resampled, y_resampled)

In [ ]:
svm.fit(X_resampled, y_resampled)

In [ ]:
rf.fit(X_resampled, y_resampled)

In [ ]:
## Model prediction
log_reg_preds = log_reg.predict(X_test)
svm_preds = svm.predict(X_test)
rf_preds = rf.predict(X_test)


In [ ]:
# Evaluate models
def evaluate_model(y_test, y_pred, model_name):
    print(f"{model_name} Classification Report:")
    print(classification_report(y_test, y_pred))
    print(f"{model_name} Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    
    # Evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    print(f"{model_name} Accuracy: {accuracy:.2f}")
    print(f"{model_name} Precision: {precision:.2f}")
    print(f"{model_name} Recall: {recall:.2f}")
    print(f"{model_name} F1-Score: {f1:.2f}")
    print("\n")

In [ ]:
# Evaluating on the test data
evaluate_model(y_test, log_reg_preds, "Logistic Regression")
evaluate_model(y_test, svm_preds, "SVM")
evaluate_model(y_test, rf_preds, "Random Forest")


In [ ]:
# Cross-validation
models = {'Logistic Regression': log_reg, 'SVM': svm, 'Random Forest': rf}

In [ ]:
for model_name, model in models.items():
    cv_scores = cross_val_score(model, X_vectorized, y, cv=5, scoring='f1_weighted',n_jobs=-1)
    print(f"{model_name} Cross-Validated F1-Score: {cv_scores.mean():.2f} ± {cv_scores.std():.2f}")

In [ ]:
# Define models and their hyperparameters for Grid Search
hyperparams = {
    'Logistic Regression': {
        'C': [0.01, 0.1, 1, 10],
        'solver': ['liblinear', 'saga']},
    'SVM': {
        'C': [1, 10],
        'kernel': ['linear'],
        'gamma': ['scale']},
    'Random Forest': {
        'n_estimators': [100, 200],
        'max_depth': [None,20]}}
best_models = {}


In [ ]:
for model_name, params in hyperparams.items():
    print(f"Optimizing {model_name}...")
    model = models[model_name]
    grid_search = GridSearchCV(model, params, cv=5, scoring='f1_weighted', n_jobs=-1)
    grid_search.fit(X_resampled, y_resampled)
    print(f"Best parameters for {model_name}: {grid_search.best_params_}")
    best_models[model_name] = grid_search.best_estimator_

In [ ]:
for model_name, model in best_models.items():
    y_pred = model.predict(X_test)
    evaluate_model(y_test, y_pred, model_name)